In [58]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score, max_error, median_absolute_error
import numpy as np
import eli5
from eli5.sklearn import PermutationImportance
from scipy.stats import norm
from sklearn.inspection import permutation_importance

In [2]:
df_all = pd.read_csv('../results/cleaned_final_1to1.csv')

In [3]:
target_cols = ['Grocery_and_Pharmacy', 'General_Retail', 'Art_and_Entertainment', 'Restaurant_and_Bar','Education', 'Healthcare']
feature_cols = ["temporal_discounting_score","loss_aversion_score",'agency_score','stringency_index',"no_health_insurance_rate","no_vehicle_household_rate", "household_income", "percent_people_own_bachelor_degrees","weighted_average_age","week","log_borough_case_count",]  # list your features here

In [4]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, 50, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [5]:
def cv_grid_search(X_train, y_train, param_grid):
    # Initialize a RandomForestRegressor
    rf = RandomForestRegressor(random_state=42)
    # Perform GridSearchCV
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
    grid_search.fit(X_train, y_train)
    # Best parameters
    # print(f"Best Parameters: {grid_search.best_params_}")
    # Use the best model found
    best_model = grid_search.best_estimator_
    return best_model

In [77]:
df_results = pd.DataFrame(columns=['Categories','Cross-Validated MSE','Mean Squared Error (MSE)','Mean Absolute Error (MAE)', 
               'R-squared', 'Mean Absolute Percentage Error (MAPE)', 'Explained Variance Score', 
               'Max Error', 'Median Absolute Error (MedAE)'])

In [66]:
def model_results(df_all, feature_cols, target_cols, param_grid, cn, df_res):
    score_importances = {}
    for idx_c in range(6):
        X = df_all[feature_cols]
        y = df_all[target_cols[idx_c]]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
        best_model0 = cv_grid_search(X, y, param_grid)
        scores = cross_val_score(best_model0, X, y, cv=5, scoring='neg_mean_squared_error')
        # score_importances[target_cols[idx_c]] = best_model0.feature_importances_
        permute_importances = permutation_importance(best_model0, X_test, y_test, n_repeats=100, random_state=42)
        score_importances[target_cols[idx_c]] = permute_importances.importances_mean
        y_pred = best_model0.predict(X)
        y_test = y
        # Assuming y_test and y_pred are the true and predicted values
        values = {'Categories':target_cols[idx_c]+cn, 'Cross-Validated MSE':scores.mean(), 'Mean Squared Error (MSE)':mean_squared_error(y_test, y_pred), 'Mean Absolute Error (MAE)':mean_absolute_error(y_test, y_pred), 'R-squared': r2_score(y_test, y_pred), 'Mean Absolute Error (MAE)':mean_absolute_error(y_test, y_pred), 'Mean Absolute Percentage Error (MAPE)':np.mean(np.abs((y_test - y_pred) / y_test)) * 100, 'Explained Variance Score':explained_variance_score(y_test, y_pred), 'Max Error':max_error(y_test, y_pred), 'Median Absolute Error (MedAE)':median_absolute_error(y_test, y_pred)}
        df_res.loc[len(df_res)] = values
    return df_res, score_importances

In [80]:
df_results, score_importance = model_results(df_all, feature_col_base, target_cols, param_grid, '_full_model',df_results)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.1s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.1s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.1s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.1s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.1s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.1s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.1s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=10, min_samples_

In [68]:
score_importance_maxmin = {}
for k, v in score_importance.items():
    # print(k, v)
    v = np.array(v)
    kr = k+'_Rank'
    score_importance_maxmin[k] = (v - np.min(v))/ (np.max(v) - np.min(v))
    score_importance_maxmin[kr] = np.argsort(np.argsort(-v)) + 1
df_importance = pd.DataFrame(score_importance_maxmin, index=feature_cols)

In [69]:
df_importance.to_csv('../results/feature_importance_full_model2.csv')

### test

In [78]:
### test
feature_col_base = ['stringency_index',"no_health_insurance_rate","no_vehicle_household_rate", "household_income", "percent_people_own_bachelor_degrees","weighted_average_age","week","log_borough_case_count",] 
for score in ["temporal_discounting_score","loss_aversion_score",'agency_score']:
    print(score)
    feature_cols_test = [score] + feature_col_base 
    # print(feature_cols_test)
    cn_i = '_with_'+score
    df_results, _ = model_results(df_all, feature_cols_test, target_cols, param_grid, cn_i, df_results)
    

[CV] END max_depth=20, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.1s
[CV] END max_depth=20, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.1s
[CV] END max_depth=10, min_samples_leaf=4, min_samples_split=2, n_estimators=200; total time=   0.2s
[CV] END max_depth=10, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.2s
[CV] END max_depth=10, min_samples_leaf=4, min_samples_split=2, n_estimators=200; total time=   0.2s
[CV] END max_depth=10, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   0.1s
[CV] END max_depth=10, min_samples_leaf=4, min_samples_split=10, n_estimators=200; total time=   0.2s
[CV] END max_depth=10, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   0.1s
[CV] END max_depth=10, min_samples_leaf=4, min_samples_split=10, n_estimators=200; total time=   0.2s
[CV] END max_depth=10, min_samples_leaf=4, min_samples_split=10, n_estimators=200; total 

In [81]:
df_results.to_csv('../results/RF_results_Vmode_with.csv', index=False)

### significant testing

In [ ]:
df_significance = pd.DataFrame(columns=['Category','Feature','Importance','Std','Z-score','P-value'])
for idx_c in range(6):
    X = df_all[feature_cols]
    y = df_all[target_cols[idx_c]]
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
    best_model0 = cv_grid_search(X, y, param_grid)
    # scores = cross_val_score(best_model0, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    y_pred = best_model0.predict(X)
    perm_importance = PermutationImportance(best_model0, n_iter=100).fit(X, y) #random_state=0, 
    # Display the feature importances
    eli5.show_weights(perm_importance, feature_names=X.columns.tolist())
    # Alternatively, you can access the feature importance values programmatically
    importances_mean = perm_importance.feature_importances_
    importances_std = perm_importance.feature_importances_std_
    # 3. Calculate z-scores for each feature's importance
    z_scores = importances_mean / importances_std
    p_values = 2 * (1 - norm.cdf(abs(z_scores)))
    # Display the importance and their corresponding standard deviations
    for i, feature in enumerate(X.columns):
        values = {}
        values = {'Category':target_cols[idx_c]}
        values['Feature'] = feature
        values['Importance'] = importances_mean[i]
        values['Std'] = importances_std[i]
        values['Z-score'] = z_scores[i]
        values['P-value'] = p_values[i]
        df_significance.loc[len(df_significance)] = values

In [14]:
df_significance['Significance'] = df_significance['P-value'].apply(lambda x: 'Significant' if x<0.05 else 'Not Significant')

In [15]:
df_significance.to_csv('../results/RF_significance_Vmode.csv', index=False)